## 🧠 Copilot Prompt

Generate a GitHub Actions workflow that automatically runs this notebook using `jupyter nbconvert --execute` and exports the HTML to `docs/validation.html`.

Ensure that `mpmath`, `sympy`, and `numpy` are installed beforehand.

If execution fails, output a warning with traceback.

**NOTE: This is the optimized version with reduced parameters for faster CI execution.**

In [ ]:
import mpmath as mp
import sympy as sp

# Reduced precision for faster execution
mp.mp.dps = 15

def f1(u): return mp.exp(-u**2/2) if abs(u) <= 3 else mp.mpf(0)
def f2(u): return mp.exp(-u**2) if abs(u) <= 2 else mp.mpf(0)
def f3(u): return (1 - u**2/4)**2 if abs(u) <= 2 else mp.mpf(0)

def fhat(f, s, lim):
    return mp.quad(lambda u: f(u) * mp.exp(s * u), [-lim, lim], maxdegree=10)

def prime_sum(f, P=50, K=10):  # Reduced parameters
    s = mp.mpf(0)
    primes = list(sp.primerange(2, 1000))[:P]  # Use smaller range
    for p in primes:
        lp = mp.log(p)
        for k in range(1, K+1):
            s += lp * f(k * lp)
    return s

def A_infty(f, sigma0=2.0, lim=3, T=10):  # Reduced T
    def integrand(t):
        s = mp.mpc(sigma0, t)
        return (mp.digamma(s/2) - mp.log(mp.pi)) * fhat(f, s, lim)
    integ = mp.quad(integrand, [-T, T], maxdegree=10) / (2 * mp.pi)
    res1 = fhat(f, mp.mpf(1), lim) / mp.mpf(1)
    return integ - res1

def zero_sum(f, N=50, lim=3):  # Much reduced N
    s = mp.mpf(0)
    for n in range(1, N+1):
        rho = mp.zetazero(n)
        s += fhat(f, mp.im(rho), lim).real
    return s

print("🔬 Running fast Riemann Hypothesis validation...")
print("Using reduced parameters: P=50, K=10, T=10, N=50")

results = []
for fname, f, lim in [('f1', f1, 2), ('f2', f2, 2)]:  # Reduced test cases and limits
    print(f"\nTesting function {fname}...")
    ps = prime_sum(f)
    ain = A_infty(f, lim=lim)
    zs = zero_sum(f, lim=lim)
    tot = ps + ain
    err = abs(tot - zs)
    rel_err = err / abs(tot) if abs(tot) > 0 else float('inf')
    
    result = f'{fname}: prime+A={float(tot.real):.6f}, zero={float(zs.real):.6f}, err={float(err):.2e}, rel_err={float(rel_err):.2e}'
    print(result)
    results.append(result)

print("\n✅ Validation completed!")
print("\n📊 Summary:")
for result in results:
    print(result)